#Ferramenta de Transcrição de Áudios com Whisper
Este projeto utiliza o poderoso modelo Whisper da OpenAI para transcrever automaticamente uma pasta inteira de arquivos de áudios curtos (como os do WhatsApp), organizando os resultados em uma tabela estruturada (.csv) pronta para análise.

**Precisa saber o que foi dito em dezenas de mensagens de voz, mas não tem tempo para ouvir todas?**

Esta ferramenta resolve esse problema, convertendo horas de áudio em texto pesquisável em questão de minutos.

##Principais Funcionalidades:

*   🤖 **Transcrição de Alta Precisão:** Utiliza os modelos de ponta da família Whisper da OpenAI, com a flexibilidade de escolher a versão que melhor se adapta às suas necessidades de velocidade e precisão.
*   🧾  **Prompt de contexto:** Você pode melhorar a performance do modelo adicionando o contexto dos áudios através de um prompt.
*   📊 **Saída Estruturada:** Organiza todas as transcrições em uma única tabela .csv, contendo o nome do arquivo, sua duração em segundos e o texto transcrito, ideal para análises futuras (como análise de sentimentos).
*   🗑️ **Limpeza Inteligente de Duplicatas:** Antes de processar, o script verifica e oferece a opção de remover arquivos duplicados com base em seu conteúdo (usando hash MD5), economizando tempo e recursos.
*   ⚙️ **Configuração Interativa:** Permite que o usuário defina parâmetros importantes durante a execução, como a escolha do modelo e a inclusão de um "prompt" para melhorar a precisão de termos específicos.

##Como Usar:


1.   **Configuração da Pasta:** Crie uma pasta no seu Google Drive (ex: audios_para_transcrever) e coloque nela todos os arquivos de áudio que deseja processar.
2.   **Execução do Notebook:** Abra o script no Google Colab, conecte-se ao seu Google Drive e execute cada célula em ordem, seguindo as instruções interativas.

Desenvolvido por: neidaniel6@gmail.com




##⚠️ Importante
A execução desta célula fará o download e a instalação desses pacotes, o que pode levar de alguns segundos a um minuto.
É fundamental aguardar a execução completa para ir para a próxima célula.

In [ ]:
# Célula 1: Instalação das Bibliotecas

# Instalar a biblioteca Whisper da OpenAI
!pip install -U openai-whisper

# Instalar a pydub para manipulação de áudio
!pip install pydub

# Garantir que o FFmpeg (dependência da pydub) esteja instalado
!apt-get -y install ffmpeg

In [ ]:
# Célula 2: Importações de bibliotecas
import whisper
import glob
import os
import hashlib
import pandas as pd
from pydub import AudioSegment
from google.colab import drive

##Conectar ao Google Drive
Caso esteja no Google Colab, esta célula é essencial para permitir que o script acesse seus arquivos de áudio. Ao executá-la, uma janela de autorização do Google aparecerá. Siga as instruções para permitir a conexão e dar ao notebook acesso à sua pasta do Google Drive.

In [ ]:
# Montar o Google Drive
drive.mount('/content/drive')

#Célula 3: Definição e Criação da Pasta de Trabalho
Esta célula define e prepara a pasta principal que usaremos para este projeto. A ideia é que você não precise editar nenhuma linha de código.

##Como Funciona:


1.   **Execute esta célula primeiro.** Ela irá criar automaticamente uma pasta em seu Google Drive no seguinte caminho: MyDrive/audios_para_transcrever.

1.   **Ação Necessária (Fora do Notebook):** Após a célula terminar de rodar, vá até a sua pasta do Google Drive, encontre a pasta audios_para_transcrever que o script acabou de criar, e mova ou copie todos os seus arquivos de áudio para dentro dela.
2.   **Próximo Passo:** Somente após seus arquivos estarem dentro desta pasta, você poderá continuar e executar a célula seguinte (Célula 3.5), que irá encontrar e listar esses arquivos.

A saída desta célula irá apenas confirmar o nome e o local da pasta que foi criada para você, garantindo que tudo está pronto para receber seus áudios.





In [ ]:
# Célula 3: Definição e Criação das Pastas de Trabalho

#---BIBLIOTECAS---
#os
#----------------

# 1. Definição da Pasta Principal
# Defina o caminho para a pasta onde seus áudios do WhatsApp estão.
# Esta pasta será usada tanto para a entrada dos áudios quanto para a saída da tabela.
work_folder = "/content/drive/MyDrive/audios_para_transcrever"

# 2. Definição do Arquivo de Saída
# O arquivo .csv com as transcrições será salvo DENTRO da pasta de trabalho.
# Usamos os.path.join para construir o caminho de forma segura.
output_csv_path = os.path.join(work_folder, "transcricoes.csv")

# 3. Criação da Pasta
# Cria a pasta de trabalho, se ela ainda não existir.
# Como a entrada e a saída estão na mesma pasta, uma única verificação é suficiente.
os.makedirs(work_folder, exist_ok=True)

# 4. Confirmação dos Caminhos Definidos
print("Configuração de pastas concluída:")
print(f"  -> Pasta de Trabalho (Entrada e Saída): {work_folder}")
print(f"  -> O arquivo de resultados será salvo em: {output_csv_path}")

##Célula 3.5: Listagem de Arquivos e Verificação de Duplicatas
Com a pasta de trabalho pronta e os áudios dentro dela, esta célula executa duas tarefas importantes: encontrar seus arquivos e realizar uma limpeza inteligente para otimizar o processo.

##O que esta célula faz:

1.   **Listagem de Arquivos:** Primeiro, o script irá "olhar" dentro da pasta audios_para_transcrever e mostrar o número total de arquivos que você colocou lá.
1.   **Verificação de Duplicatas:** Em seguida, ele analisa o conteúdo de cada arquivo para encontrar duplicatas exatas. Isso é muito útil caso você tenha copiado o mesmo áudio mais de uma vez, mesmo que os nomes dos arquivos sejam diferentes.
3.   **Limpeza Opcional:** Se duplicatas forem encontradas, o script irá parar e perguntar se você deseja apagá-las. Esta é a sua chance de fazer uma limpeza automática e evitar transcrever o mesmo áudio várias vezes.


## Sua Ação
A única interação necessária aqui será responder s (sim) ou n (não) à pergunta sobre apagar os arquivos, caso o script encontre alguma duplicata.

Ao final da execução, a saída confirmará o número final de arquivos únicos que serão enviados para a transcrição na próxima etapa.



In [ ]:
# Célula 3.5: Listagem de Arquivos e Verificação de Duplicatas

#---BIBLIOTECAS---
#os
#glob
#hashlib
#----------------

# 1. Listagem dos Arquivos de Áudio
print("--- Listando arquivos de áudio ---")
# Lista todos os arquivos na pasta de trabalho definida na Célula 3
audio_files = glob.glob(os.path.join(work_folder, "*.*"))
print(f"Total de arquivos encontrados inicialmente: {len(audio_files)}")

# 2. Função para Calcular Hash MD5
def calculate_md5(file_path):
    """Calcula o hash MD5 de um arquivo de forma eficiente."""
    md5_hash = hashlib.md5()
    with open(file_path, "rb") as f:
        # Lê o arquivo em pedaços para não sobrecarregar a memória
        for byte_block in iter(lambda: f.read(4096), b""):
            md5_hash.update(byte_block)
    return md5_hash.hexdigest()

# 3. Verificação de Duplicatas
print("\n--- Iniciando verificação de arquivos duplicados (usando MD5) ---")
hashes = {}
duplicates = []

for audio_path in audio_files:
    # Ignorar subdiretórios, verificar apenas arquivos
    if not os.path.isfile(audio_path):
        continue
    try:
        file_hash = calculate_md5(audio_path)
        if file_hash in hashes:
            duplicates.append(audio_path)
        else:
            hashes[file_hash] = audio_path
    except Exception as e:
        print(f"  -> ERRO ao calcular hash para {os.path.basename(audio_path)}: {e}")

if duplicates:
    print(f"\nForam encontrados {len(duplicates)} arquivos duplicados.")
    user_choice = input("Deseja apagar os arquivos duplicados? (s/n): ").lower()

    if user_choice == 's':
        deleted_count = 0
        for dup_path in duplicates:
            try:
                os.remove(dup_path)
                print(f"  - Apagado: {os.path.basename(dup_path)}")
                deleted_count += 1
            except Exception as e:
                print(f"  - ERRO ao apagar {os.path.basename(dup_path)}: {e}")
        print(f"\n{deleted_count} arquivos duplicados foram apagados.")
        # Atualiza a lista de arquivos para conter apenas os caminhos dos arquivos únicos
        audio_files = list(hashes.values())
    else:
        print("\nNenhum arquivo duplicado foi apagado. A lista de arquivos pode conter duplicatas.")
else:
    print("Nenhum arquivo duplicado foi encontrado.")

# 4. Confirmação Final
print("\n-------------------------------------------------------------")
print(f"Verificação de duplicatas concluída.")
print(f"Número final de arquivos a serem processados: {len(audio_files)}")
print("-------------------------------------------------------------")

# Célula 4: Escolha e Carregamento do Modelo de IA
Esta célula carrega o modelo de IA Whisper. Ao executá-la, um menu interativo aparecerá para você escolher a versão do modelo, permitindo balancear velocidade e precisão.

## Guia Rápido das Opções:
*   Para Velocidade: Use tiny (1) ou base (2).
*   Para Equilíbrio (Recomendado): Use small (3).
*   Para Precisão Máxima: Use medium (4) ou large (5).

## ⚠️ Aviso Sobre Memória e Áudios Longos
Os modelos maiores (medium e large) consomem muita memória e podem causar erros se usados para transcrever áudios muito longos (ex: acima de 15 minutos). Para áudios curtos, como os do WhatsApp, eles costumam funcionar bem.

Após sua escolha, o modelo será carregado. Isso pode levar alguns minutos na primeira vez.

In [ ]:
# Célula 4: Carregar o Modelo Whisper (com seleção interativa)

# Mapeia a escolha do usuário para o nome do modelo
model_options = {
    '1': 'tiny',
    '2': 'base',
    '3': 'small',
    '4': 'medium',
    '5': 'large'
}

# Loop para garantir que o usuário escolha uma opção válida
while True:
    print("--- Escolha o modelo do Whisper para carregar ---")
    print("Modelos menores são mais rápidos, modelos maiores são mais precisos.")
    print("Recomendação para Colab padrão: 'base' ou 'small'.\n")
    print("1: tiny (mais rápido, menor precisão, ideal para testes)")
    print("2: base (bom equilíbrio entre velocidade e precisão)")
    print("3: small (mais preciso, um pouco mais lento)")
    print("4: medium (alta precisão, significativamente mais lento)")
    print("5: large (precisão máxima, MUITO lento, pode causar erro de memória)")

    user_choice = input("\nDigite o número do modelo desejado (1-5): ")

    if user_choice in model_options:
        model_size = model_options[user_choice]
        print(f"\nVocê selecionou o modelo: '{model_size}'. Carregando, isso pode levar um tempo...")
        break  # Sai do loop, pois a escolha é válida
    else:
        print("\nOpção inválida! Por favor, escolha um número de 1 a 5.\n")

# Carrega o modelo escolhido pelo usuário
try:
    model = whisper.load_model(model_size)
    print(f"\n✅ Modelo Whisper '{model_size}' carregado com sucesso!")
except Exception as e:
    print(f"\n❌ ERRO ao carregar o modelo: {e}")
    print("Se estiver usando os modelos 'medium' ou 'large', pode ser um erro de falta de memória.")

# Célula 5: Executar a Transcrição dos Áudios
Esta é a célula principal, onde a mágica acontece. Ela irá percorrer cada um dos seus arquivos de áudio e usar o modelo Whisper para convertê-los em texto.

Ao executar, o script primeiro perguntará se você deseja usar um Prompt Inicial. Esta é uma ferramenta opcional, mas poderosa, para melhorar a precisão da IA, especialmente na grafia de nomes, jargões ou termos técnicos.

## Dica Avançada: Como Criar um Prompt Eficaz com IA
Se você não tem certeza do que escrever no prompt, use esta técnica para que uma IA crie o prompt para você:

1.   **Faça uma Amostra:** Mova alguns poucos áudios (3 a 5) que representem bem o seu conteúdo para a pasta audios_para_transcrever.

2.   **Transcreva a Amostra:** Rode esta célula e responda n (não) à pergunta sobre o prompt.
3.   **Peça Ajuda a uma LLM:** Copie as transcrições geradas e cole em uma LLM de sua preferência (Gemini, ChatGPT, etc.), usando um comando como:

*"Analise estas transcrições e crie um prompt de contexto para o modelo Whisper, listando os principais nomes, jargões ou locais que aparecem."*

4.   **Execute a Versão Final:** Coloque todos os seus áudios de volta na pasta. Rode a Célula 5 novamente, mas desta vez responda s (sim) e use o prompt que a LLM gerou para você.

Após a configuração, o script iniciará o processo, mostrando o progresso a cada arquivo. Este processo pode levar tempo, dependendo da quantidade de áudios e do modelo escolhido.

In [ ]:
# Célula 5: Processar e Transcrever os Áudios

# Lista para armazenar os resultados
transcription_data = []

# --- CONFIGURAÇÕES DE TRANSCRIÇÃO ---

# 1. Lógica para o Prompt Interativo
prompt_inicial = "" # Inicializa como uma string vazia
while True:
    usar_prompt = input("Deseja inserir um prompt inicial para melhorar a precisão? (s/n): ").lower()
    if usar_prompt in ['s', 'n']:
        break
    else:
        print("Opção inválida. Por favor, digite 's' para sim ou 'n' para não.")

if usar_prompt == 's':
    prompt_inicial = input("Digite o prompt (ex: nomes, lugares ou jargões que podem aparecer nos áudios):\n")

# 2. Outras configurações
tamanho_do_beam = 10 # Padrão é 5. Aumentar pode melhorar a precisão, mas torna mais lento.
# ------------------------------------

print("\nIniciando o processo de transcrição com as configurações definidas...")

# O loop usa a variável 'audio_files',
# que já foi definida e limpa de duplicatas na Célula 3.5.
for i, audio_path in enumerate(audio_files):
    file_name = os.path.basename(audio_path)
    print(f"Processando ({i+1}/{len(audio_files)}): {file_name}")

    try:
        # Obter a duração do áudio
        audio = AudioSegment.from_file(audio_path)
        duration_seconds = len(audio) / 1000.0

        # Transcrever o áudio com parâmetros ajustados
        result = model.transcribe(
            audio_path,
            language='pt',
            initial_prompt=prompt_inicial,
            beam_size=tamanho_do_beam
        )
        transcription = result["text"]

        # Adicionar os dados à nossa lista
        transcription_data.append({
            "Arquivo": file_name,
            "Duração (s)": duration_seconds,
            "Transcrição": transcription
        })

        print(f"  -> Transcrição: {transcription[:80]}...")

    except Exception as e:
        print(f"  -> ERRO ao processar o arquivo {file_name}: {e}")

print("\nProcesso de transcrição concluído!")

# Célula 6: Criar, Formatar e Salvar a Tabela de Transcrições
Esta é a última etapa do nosso pipeline. Ela pega todas as transcrições geradas na célula anterior e as organiza em um arquivo final, limpo e pronto para uso.

## O que esta célula faz:

1.   Cria a Tabela: Organiza todas as informações coletadas (nome do arquivo, duração e texto) em uma tabela estruturada.
2.   Salva o Resultado: Grava a tabela completa em um arquivo chamado transcricoes.csv dentro da sua pasta de trabalho (**audios_para_transcrever**) no Google Drive.

## O Resultado Final
Ao executar, a saída da célula mostrará as primeiras linhas da sua tabela final, permitindo que você visualize o resultado imediatamente.

**O arquivo transcricoes.csv estará na sua pasta**, pronto para ser baixado ou aberto diretamente no Excel ou no Planilhas Google, com toda a acentuação e caracteres especiais preservados corretamente.

Parabéns, seu processo de transcrição automatizada está concluído!

In [ ]:
# Célula 6: Criar, Formatar e Salvar a Tabela de Transcrições
#---BIBLIOTECAS---
#os
#pandas
#----------------
# 1. Criar um DataFrame do pandas com os dados coletados
df = pd.DataFrame(transcription_data)

# 2. Formatar a coluna de duração para segundos inteiros
if not df.empty and 'Duração (s)' in df.columns:
    df['Duração (s)'] = df['Duração (s)'].round().astype(int)

# --- Opção de Salvamento ---
print("\n--- Salvando a Tabela de Resultados ---")
print("O arquivo será salvo no formato .csv com a codificação 'utf-8-sig',")
print("que garante a máxima compatibilidade com acentuação no Excel e Planilhas Google.")

# 3. Salvar o DataFrame final em um arquivo .csv
# A variável 'output_csv_path' foi definida na Célula 3.
df.to_csv(output_csv_path, index=False, encoding='utf-8-sig')

print(f"\n✅ Tabela de transcrições salva com sucesso em: {output_csv_path}")

# 4. Exibir as 10 primeiras linhas da tabela final para verificação
print("\nAmostra dos resultados finais:")
display(df.head(10))